In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
base_path = '../../'
sys.path.append(base_path)

In [26]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data as data
import numpy as np
from tqdm import tqdm

# 3D U-Net with a small parameter count
class SmallUNet3D(nn.Module):
    def __init__(self, in_channels=1, out_channels=5, base_filters=16):
        super(SmallUNet3D, self).__init__()
        self.enc1 = nn.Conv3d(in_channels, base_filters, kernel_size=3, padding=1)
        self.enc2 = nn.Conv3d(base_filters, base_filters * 2, kernel_size=3, padding=1)
        
        self.pool = nn.MaxPool3d(2)
        self.upsample = nn.Upsample(scale_factor=2, mode='trilinear', align_corners=True)
        
        self.dec1 = nn.Conv3d(base_filters * 2, base_filters, kernel_size=3, padding=1)
        self.dec2 = nn.Conv3d(base_filters, out_channels, kernel_size=1)  # Final regression output

        self.act = nn.ReLU()
    
    def forward(self, x):
        e1 = self.act(self.enc1(x))
        e2 = self.act(self.enc2(self.pool(e1)))
        
        d1 = self.act(self.dec1(self.upsample(e2)))
        d2 = self.dec2(d1)  # No activation for regression
        
        return d2




# Initialize model and train
model = SmallUNet3D()


In [18]:
from src.pyvista_flow_field_dataset import PyvistaFlowFieldDataset
from src.voxel_flow_field_dataset import VoxelFlowFieldDataset, VoxelFlowFieldDatasetConfig
ds_pv = PyvistaFlowFieldDataset.load_from_huggingface(num_samples=3)
ds_voxel = VoxelFlowFieldDataset('datasets/voxels',VoxelFlowFieldDatasetConfig(ds_pv))
#ds_voxel = VoxelFlowFieldDataset('datasets/voxels')
ds_voxel.normalize()

ValueError: Error getting the list of files in repository 'peteole/CoolMucSmall': (MaxRetryError('HTTPSConnectionPool(host=\'huggingface.co\', port=443): Max retries exceeded with url: /api/datasets/peteole/CoolMucSmall/tree/main?recursive=True&expand=False (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x17e203d50>: Failed to resolve \'huggingface.co\' ([Errno 8] nodename nor servname provided, or not known)"))'), '(Request ID: 0a4141a8-33b2-4c66-823f-158fa69a5e0c)')

In [19]:

import torch.utils.data as data
dataloader = data.DataLoader(ds_voxel.get_default_loadable_dataset(), batch_size=3, shuffle=True)

In [29]:

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
criterion = nn.MSELoss()
lr=1e-3
epochs=100
optimizer = optim.Adam(model.parameters(), lr=lr)

for epoch in range(epochs):
    model.train()
    epoch_loss = 0

    for x,y in tqdm(dataloader, desc=f"Epoch {epoch+1}/{epochs}"):
        x, y = x.to(device), y.to(device)
        x=x.float().unsqueeze(-1)
        x = x.permute(0, 4, 1, 2, 3)  # (N, D, H, W, C) → (N, C, D, H, W)
        y = y.permute(0, 4, 1, 2, 3)
        optimizer.zero_grad()
        outputs = model(x)
        loss = criterion(outputs, y)
        loss.backward()
        optimizer.step()

        epoch_loss += loss.item()

    print(f"Epoch {epoch+1}, Loss: {epoch_loss / len(dataloader)}")

Epoch 1/100: 100%|██████████| 1/1 [00:00<00:00,  2.83it/s]


Epoch 1, Loss: 0.6820264458656311


Epoch 2/100: 100%|██████████| 1/1 [00:00<00:00,  3.29it/s]


Epoch 2, Loss: 2.4455931186676025


Epoch 3/100: 100%|██████████| 1/1 [00:00<00:00,  3.19it/s]


...

Epoch 98, Loss: 0.6816698908805847


Epoch 99/100: 100%|██████████| 1/1 [00:00<00:00,  1.76it/s]


Epoch 99, Loss: 0.6813727021217346


Epoch 100/100: 100%|██████████| 1/1 [00:00<00:00,  1.98it/s]

Epoch 100, Loss: 0.6810876727104187
